This notebook aims to create a `warp_matrices` file which is an affine transformation that can align the captures of the Dual-MX camera system. This `warp_matrices` file will be applied to all the flights

1. Find a capture where there are sufficient features in the scene
2. Make sure that the height of the capture > 35 m AGL


For best alignment results it's good to select a capture which has features which visible in all bands.  Man-made objects such as cars, roads, and buildings tend to work very well, while captures of only repeating crop rows tend to work poorly.  Remember, once a good transformation has been found for flight, it can be generally be applied across all of the images.

It's also good to use an image for alignment which is taken near the same level above ground as the rest of the flights. Above approximately 35m AGL, the alignement will be consistent. However, if images taken closer to the ground are used, such as panel images, the same alignment transformation will not work for the flight data.  